# Import Libraries and Data

In [36]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import TruncatedSVD
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from joblib import dump, load

%run -i 'functions/model_eval.py'

In [4]:
modeling_df = pd.read_csv('/Users/will4856/Downloads/modeling_df.csv', index_col='Unnamed: 0')
modeling_df.head()

,Title,year,production_budget,domestic_gross,worldwide_gross,profit,ROI,script,Metascore,imdbRating,...,Metascore_scaled,imdbRating_scaled,success_metric,cleaned_scripts,success_failure,word_cloud_scripts,word_count,WPM,unique_word_count,total_vocab_diversity
0,Pirates of the Caribbean: On Stranger Tides,2011,379000000,241063875,1045663875,666663875,1.76,Sub.Re-Edit.by. :: npdv.indoheroes[at]gmail.co...,45.0,6.6,...,0.408602,0.649351,1.38,sub edit npdv indoheroes gmail com advertise p...,0,"['sub', 'edit', 'npdv', 'indoheroes', 'gmail',...",3992,29.4,1547,0.39
1,Avengers: Age of Ultron,2015,365000000,459005868,1396099202,1031099202,2.82,(DISTANT EXPLOSION) STRUCKER ON PA: Report to ...,66.0,7.3,...,0.634409,0.740260,1.56,distant explosion strucker pa report stations ...,1,"['distant', 'explosion', 'strucker', 'pa', 're...",5690,40.4,1869,0.33
2,Justice League,2017,300000000,229024295,655945209,355945209,1.19,"There he is! Oh, sorry. Superman, Superman, ca...",45.0,6.4,...,0.408602,0.623377,1.34,oh sorry superman superman ask questions podca...,0,"['oh', 'sorry', 'superman', 'superman', 'ask',...",3255,27.1,1350,0.41
3,Spectre,2015,300000000,200074175,879620923,579620923,1.93,"Where are you going? I won't be long. Welcome,...",60.0,6.8,...,0.569892,0.675325,1.45,going long welcome signor soiarra trust pleasa...,1,"['going', 'long', 'welcome', 'signor', 'soiarr...",2937,19.8,1245,0.42
4,Batman v Superman: Dawn of Justice,2016,263000000,330360194,872395091,609395091,2.32,There was a time above. A time before. There w...,44.0,6.5,...,0.397849,0.636364,1.36,time time perfect things diamond absolutes thi...,0,"['time', 'time', 'perfect', 'things', 'diamond...",4451,29.5,1748,0.39


In [5]:
#modeling_df.to_csv('/Users/will4856/Desktop/script_model/modeling_df.csv')

# Data Pre-Processing

In [6]:
modeling_df = modeling_df.loc[modeling_df['cleaned_scripts'].notna()]

In [7]:
modeling_df['cleaned_scripts'] = modeling_df['cleaned_scripts'].map(lambda x: ''.join(x))

In [8]:
modeling_df

,Title,year,production_budget,domestic_gross,worldwide_gross,profit,ROI,script,Metascore,imdbRating,...,Metascore_scaled,imdbRating_scaled,success_metric,cleaned_scripts,success_failure,word_cloud_scripts,word_count,WPM,unique_word_count,total_vocab_diversity
0,Pirates of the Caribbean: On Stranger Tides,2011,379000000,241063875,1045663875,666663875,1.76,Sub.Re-Edit.by. :: npdv.indoheroes[at]gmail.co...,45.0,6.6,...,0.408602,0.649351,1.38,sub edit npdv indoheroes gmail com advertise p...,0,"['sub', 'edit', 'npdv', 'indoheroes', 'gmail',...",3992,29.4,1547,0.39
1,Avengers: Age of Ultron,2015,365000000,459005868,1396099202,1031099202,2.82,(DISTANT EXPLOSION) STRUCKER ON PA: Report to ...,66.0,7.3,...,0.634409,0.740260,1.56,distant explosion strucker pa report stations ...,1,"['distant', 'explosion', 'strucker', 'pa', 're...",5690,40.4,1869,0.33
2,Justice League,2017,300000000,229024295,655945209,355945209,1.19,"There he is! Oh, sorry. Superman, Superman, ca...",45.0,6.4,...,0.408602,0.623377,1.34,oh sorry superman superman ask questions podca...,0,"['oh', 'sorry', 'superman', 'superman', 'ask',...",3255,27.1,1350,0.41
3,Spectre,2015,300000000,200074175,879620923,579620923,1.93,"Where are you going? I won't be long. Welcome,...",60.0,6.8,...,0.569892,0.675325,1.45,going long welcome signor soiarra trust pleasa...,1,"['going', 'long', 'welcome', 'signor', 'soiarr...",2937,19.8,1245,0.42
4,Batman v Superman: Dawn of Justice,2016,263000000,330360194,872395091,609395091,2.32,There was a time above. A time before. There w...,44.0,6.5,...,0.397849,0.636364,1.36,time time perfect things diamond absolutes thi...,0,"['time', 'time', 'perfect', 'things', 'diamond...",4451,29.5,1748,0.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3004,Princess Mononoke,2005,20000000,2374107,150345863,130345863,6.52,"[In ancient times, the land, lay covered i...",76.0,8.4,...,0.741935,0.883117,1.80,ancient times land lay covered forests ages lo...,1,"['ancient', 'times', 'land', 'lay', 'covered',...",4583,34.2,1235,0.27
3005,Bad Grandpa,2006,15000000,102003019,160903019,145903019,9.73,"[- Is it going?, - Yes., Okay., Leave a messa...",54.0,6.5,...,0.505376,0.636364,1.39,going yes okay leave message irving zisman fuc...,0,"['going', 'yes', 'okay', 'leave', 'message', '...",5659,61.5,1526,0.27
3006,High School Musical 3,2000,11000000,90559416,274392880,263392880,23.94,"[Let's go!, - I'm open, I'm open., - I don't s...",57.0,4.8,...,0.537634,0.415584,1.07,let go open open see take time run run go go l...,0,"['let', 'go', 'open', 'open', 'see', 'take', '...",5185,46.3,1289,0.25
3007,A Nightmare on Elm Street 5: The Dream Child,2014,6000000,22168359,22168359,16168359,2.69,"[Sixty-eight... sixty-nine..., seventy... seve...",54.0,5.2,...,0.505376,0.467532,1.12,sixty eight sixty nine seventy seventy one com...,0,"['sixty', 'eight', 'sixty', 'nine', 'seventy',...",2095,23.5,828,0.40


## Vectorizing

In [9]:
vectorizer = TfidfVectorizer()

In [10]:
X = modeling_df['cleaned_scripts']
y = modeling_df['success_failure']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.33)

In [12]:
X_vect = vectorizer.fit_transform(X)
X_train_tfidf = vectorizer.transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

## Dimensionality Reduction with TruncatedSVD

In [13]:
tsvd = TruncatedSVD()

In [14]:
tsvd.fit(X_vect)
X_train_tsvd = tsvd.transform(X_train_tfidf)
X_test_tsvd = tsvd.transform(X_test_tfidf)

# Classification Modeling

## XGBoosted Classifier

In [15]:
xgbc_clf = XGBClassifier()

In [16]:
model_eval(xgbc_clf)

              precision    recall  f1-score   support

           0       0.55      0.62      0.58       466
           1       0.61      0.54      0.57       513

    accuracy                           0.58       979
   macro avg       0.58      0.58      0.58       979
weighted avg       0.58      0.58      0.58       979

[[290 176]
 [237 276]]
Training Accuracy Score ->  67.5226586102719
Test Accuracy Score ->  57.81409601634321


In [17]:
params_xgboost = {'max_depth': range(2,10,1),
         'n_estimators': range(60,220,40),
         'learning_rate': [0.1, 0.01, 0.05]}
estimator_xgboost = XGBClassifier(
    objective= 'binary:logistic',
    n_jobs=-1,
    seed=42, 
)
grid_search_xgboost = GridSearchCV(
    estimator=estimator_xgboost,
    param_grid=params_xgboost,
    scoring = 'accuracy',
    n_jobs = -1,
    cv = 10,
    verbose=True)

In [18]:
grid_search_xgboost.fit(X_train_tsvd, y_train)

Fitting 10 folds for each of 96 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   17.5s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   32.2s
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed:   40.2s finished


GridSearchCV(cv=10, estimator=XGBClassifier(n_jobs=-1, seed=42), n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.01, 0.05],
                         'max_depth': range(2, 10),
                         'n_estimators': range(60, 220, 40)},
             scoring='accuracy', verbose=True)

In [19]:
grid_search_xgboost.best_params_

{'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 60}

In [20]:
xgbc_clf = XGBClassifier(objective = 'binary:logistic', learning_rate = 0.01, max_depth = 2, n_estimators = 100)

In [21]:
model_eval(xgbc_clf)

              precision    recall  f1-score   support

           0       0.55      0.59      0.57       466
           1       0.60      0.56      0.58       513

    accuracy                           0.58       979
   macro avg       0.58      0.58      0.58       979
weighted avg       0.58      0.58      0.58       979

[[277 189]
 [227 286]]
Training Accuracy Score ->  61.17824773413897
Test Accuracy Score ->  57.507660878447396


## Support Vector Machine Classifier

In [22]:
svc_clf = SVC()

In [23]:
model_eval(svc_clf)

              precision    recall  f1-score   support

           0       0.56      0.62      0.59       466
           1       0.61      0.55      0.58       513

    accuracy                           0.58       979
   macro avg       0.59      0.58      0.58       979
weighted avg       0.59      0.58      0.58       979

[[289 177]
 [231 282]]
Training Accuracy Score ->  60.87613293051359
Test Accuracy Score ->  58.324821246169556


In [24]:
params_svc = {'C': [0.1, 1, 10, 100], 'gamma':[1,0.1,0.01,0.001]}
estimator_svc = SVC(kernel='linear',
                    class_weight='balanced')
grid_search_svc = GridSearchCV(estimator=estimator_svc, 
                              param_grid=params_svc,
                              scoring = 'accuracy',
                              n_jobs = -1,
                              cv = 10,
                              verbose = True)

In [25]:
grid_search_svc.fit(X_train_tsvd, y_train)

Fitting 10 folds for each of 16 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:    2.7s finished


GridSearchCV(cv=10, estimator=SVC(class_weight='balanced', kernel='linear'),
             n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100],
                         'gamma': [1, 0.1, 0.01, 0.001]},
             scoring='accuracy', verbose=True)

In [26]:
grid_search_svc.best_params_

{'C': 1, 'gamma': 1}

In [27]:
svc_clf = SVC(kernel = 'linear', C = 1, gamma = 1, class_weight='balanced')

In [28]:
model_eval(svc_clf)

              precision    recall  f1-score   support

           0       0.56      0.49      0.52       466
           1       0.58      0.65      0.62       513

    accuracy                           0.58       979
   macro avg       0.57      0.57      0.57       979
weighted avg       0.57      0.58      0.57       979

[[227 239]
 [177 336]]
Training Accuracy Score ->  58.006042296072515
Test Accuracy Score ->  57.507660878447396


## Multinomial Naive Bayes Classifier

In [29]:
nbc = MultinomialNB()

In [30]:
c_vectorizer = CountVectorizer()

In [31]:
Xc_vect = c_vectorizer.fit_transform(X)

In [32]:
X_train_cvect, X_test_cvect, y_train, y_test = train_test_split(Xc_vect, y, test_size = .5)

In [33]:
nbc.fit(X_train_cvect, y_train)

MultinomialNB()

In [34]:
y_test_preds = nbc.predict(X_test_cvect)
y_train_preds = nbc.predict(X_train_cvect)
print(classification_report(y_test, y_test_preds))
print(confusion_matrix(y_test, y_test_preds))
print("Training Accuracy Score -> ",accuracy_score(y_train, y_train_preds)*100)
print("Test Accuracy Score -> ",accuracy_score(y_test, y_test_preds)*100)

              precision    recall  f1-score   support

           0       0.58      0.86      0.69       741
           1       0.72      0.37      0.49       742

    accuracy                           0.61      1483
   macro avg       0.65      0.61      0.59      1483
weighted avg       0.65      0.61      0.59      1483

[[635 106]
 [466 276]]
Training Accuracy Score ->  86.30229419703103
Test Accuracy Score ->  61.42953472690492


## Dump Model to PKL File to Deploy

In [29]:
dump(nbc, 'NB_script_model.pkl')

['NB_script_model.pkl']

# Introduce More Features to Model

In [38]:
X = modeling_df[['cleaned_scripts','word_count', 'WPM', 'unique_word_count', 'total_vocab_diversity']]
y = modeling_df['success_failure']

## Train/Test Split, Vectorize, Dimensionality Reduction

In [39]:
vectorizer = TfidfVectorizer()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.25)
X_vect = vectorizer.fit_transform(X)
X_train_tfidf = vectorizer.transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)
tsvd = TruncatedSVD()
tsvd.fit(X_vect)
X_train_tsvd = tsvd.transform(X_train_tfidf)
X_test_tsvd = tsvd.transform(X_test_tfidf)

In [40]:
xgbc_clf = XGBClassifier()

In [41]:
model_eval(xgbc_clf)

XGBoostError: [12:25:33] src/objective/regression_obj.cu:65: Check failed: preds.Size() == info.labels_.Size() (5 vs. 2223) : labels are not correctly providedpreds.size=5, label.size=2223
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x0000001a1f4a2e19 dmlc::LogMessageFatal::~LogMessageFatal() + 57
  [bt] (1) 2   libxgboost.dylib                    0x0000001a1f529a36 xgboost::obj::RegLossObj<xgboost::obj::LogisticClassification>::GetGradient(xgboost::HostDeviceVector<float> const&, xgboost::MetaInfo const&, int, xgboost::HostDeviceVector<xgboost::detail::GradientPairInternal<float> >*) + 646
  [bt] (2) 3   libxgboost.dylib                    0x0000001a1f49ea9e xgboost::LearnerImpl::UpdateOneIter(int, xgboost::DMatrix*) + 1278
  [bt] (3) 4   libxgboost.dylib                    0x0000001a1f4bf34c XGBoosterUpdateOneIter + 172
  [bt] (4) 5   libffi.6.dylib                      0x000000010f09f884 ffi_call_unix64 + 76
  [bt] (5) 6   ???                                 0x00